In [ ]:
#####################################
# This creates figure
# NEPac-NAtl-ana.png and 
# in paper
#####################################
import sys
# import modules in other directories
sys.path.append('../Utils/')

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import copy
import importlib
import get_lens1_rcp85 as lens1
import sst_biases_2018pub as sstbias

import trax_util as trx
import ibtracs as IBT
import ts_w_trx_joint as ttJ
import PlotUtil as Pu

importlib.reload( Pu )
importlib.reload( sstbias )
importlib.reload( lens1 )
importlib.reload( trx )
importlib.reload( IBT )
importlib.reload( ttJ )


In [ ]:
def plot_BasinPoly(ax,BasinPoly):
    import matplotlib.patches as patches
    from shapely.geometry import Polygon
    ###########################
    polygon = Polygon( BasinPoly )
    buffered_polygon = polygon.buffer(buffer_distance)
    Outer_polygon = patches.Polygon( BasinPoly , closed=True, edgecolor='red', facecolor='none', transform=projData )
    ax.add_patch(Outer_polygon)
    Inner_polygon = patches.Polygon(list(buffered_polygon.exterior.coords), edgecolor='red', facecolor='none', linestyle='--',transform=ccrs.PlateCarree())
    ax.add_patch(Inner_polygon)


In [ ]:
#return BACE_stack,ts_stack,BACE_pd_stack,had_stack,landf,lat,lon
# BACE_stack, ts_stack, BACE_pd_stack, had_stack, landf, lat, lon = ttJ.trx_w_ts()
BACE_stack, ts_stack, BACE_pd_stack, had_stack, landf, lat, lon , sBACE_stack = ttJ.trx_w_ts(xNEPacific=True)

### Obs
ibx=IBT.readtrx()
BACE_ibx,YrI,MonI =trx.basinACEmonth(wind=ibx.wind,basin=ibx.basin,year=ibx.year,month=ibx.month,power_wind=1)


## AT this point you've read in the LENS TS data for 2070-2099 with pre-pended TS from an ealrier CESM1 coupled run
## Also have IBtrACS and HADLEY SSTs

In [ ]:
####
# Deep Copies
####
ts_stack_O = copy.deepcopy( ts_stack )
had_stack_O = copy.deepcopy( had_stack )

In [ ]:
#### 
# Deseasonalize ...
###
nens,nYr,nMo,ny,nx = np.shape(ts_stack)
ts_stack_A = np.zeros( (nens,nMo,ny,nx ) )
for im in np.arange( 12 ):
    for iy in np.arange( nYr ):
        ts_stack_A[:,im,:,:] = ts_stack_A[:,im,:,:] + ts_stack[:,iy,im,:,:] 
ts_stack_A = ts_stack_A / nYr
for im in np.arange( 12 ):
    for iy in np.arange( nYr ):
        ts_stack[:,iy,im,:,:] = ts_stack[:,iy,im,:,:] - ts_stack_A[:,im,:,:]


nens,nYr,nMo,ny,nx = np.shape(had_stack)
had_stack_A = np.zeros( (nens,nMo,ny,nx ) )
for im in np.arange( 12 ):
    for iy in np.arange( nYr ):
        had_stack_A[:,im,:,:] = had_stack_A[:,im,:,:] + had_stack[:,iy,im,:,:] 
had_stack_A = had_stack_A / nYr
for im in np.arange( 12 ):
    for iy in np.arange( nYr ):
        had_stack[:,iy,im,:,:] = had_stack[:,iy,im,:,:] - had_stack_A[:,im,:,:]




In [ ]:
def scale_polygon(coords, scale=1.05):
    cent_x = sum(x for x, y in coords) / len(coords)
    cent_y = sum(y for x, y in coords) / len(coords)
    
    scaled_coords = [(cent_x + (x - cent_x) * scale, cent_y + (y - cent_y) * scale) for x, y in coords]
    
    return scaled_coords


In [ ]:
#### 
# Deseasonalize ...
# using ensemble mean ...
###
nens,nYr,nMo,ny,nx = np.shape(ts_stack_O)
ts_stack_A = np.zeros( ( nMo,ny,nx ) )
ts_stack_E = np.zeros(( nens,nYr,nMo,ny,nx ))
for im in np.arange( 12 ):
    for iy in np.arange( nYr ):
        for ie in np.arange( nens ):
            ts_stack_A[im,:,:] = ts_stack_A[im,:,:] + ts_stack_O[ie,iy,im,:,:] 
ts_stack_A = ts_stack_A / ( nYr*nens )
for im in np.arange( 12 ):
    for iy in np.arange( nYr ):
        for ie in np.arange( nens ):
            ts_stack_E[ie,iy,im,:,:] = ts_stack_O[ie,iy,im,:,:] - ts_stack_A[im,:,:]


"""
nens,nYr,nMo,ny,nx = np.shape(had_stack)
had_stack_A = np.zeros( (nens,nMo,ny,nx ) )
for im in np.arange( 12 ):
    for iy in np.arange( nYr ):
        had_stack_A[:,im,:,:] = had_stack_A[:,im,:,:] + had_stack[:,iy,im,:,:] 
had_stack_A = had_stack_A / nYr
for im in np.arange( 12 ):
    for iy in np.arange( nYr ):
        had_stack[:,iy,im,:,:] = had_stack[:,iy,im,:,:] - had_stack_A[:,im,:,:]

"""


In [ ]:
###
allyear=np.arange(12)+1
NHseason=[5,6,7,8,9,10,11,12]
SHseason=[1,2,3,4,5,10,11,12]
dont_scale=True
ACEo_NAtl_Fu, dts_over_NAtl_Fu, dts_under_NAtl_Fu = \
    ttJ.sst_composite_1 (basin=1, BACE_stack=BACE_stack, ts_stack=ts_stack,
                    landf=landf, months=NHseason,dont_scale = dont_scale) #[5,6,7,8,9,10,11] )
ACEo_SInd_Fu, dts_over_SInd_Fu, dts_under_SInd_Fu = \
    ttJ.sst_composite_1 (basin=7, BACE_stack=BACE_stack, ts_stack=ts_stack,
                    landf=landf, months=SHseason,dont_scale = dont_scale) #[1,2,3,4,5,10,11,12] )
ACEo_NEPac_Fu, dts_over_NEPac_Fu, dts_under_NEPac_Fu = \
    ttJ.sst_composite_1 (basin=4, BACE_stack=BACE_stack, ts_stack=ts_stack,
                    landf=landf, months=NHseason,dont_scale = dont_scale) #[5,6,7,8,9,10,11] )
ACEo_xNEPac_Fu, dts_over_xNEPac_Fu, dts_under_xNEPac_Fu = \
    ttJ.sst_composite_1 (basin=2, BACE_stack=sBACE_stack, ts_stack=ts_stack,
                    landf=landf, months=NHseason,dont_scale = dont_scale) #[5,6,7,8,9,10,11] )


In [ ]:
###
allyear=np.arange(12)+1
NHseason=[5,6,7,8,9,10,11,12]
SHseason=[1,2,3,4,5,10,11,12]
dont_scale=True
ACEo_NAtl_Fu_E, dts_over_NAtl_Fu_E, dts_under_NAtl_Fu_E = \
    ttJ.sst_composite_1 (basin=1, BACE_stack=BACE_stack, ts_stack=ts_stack_E,
                    landf=landf, months=NHseason,dont_scale = dont_scale) #[5,6,7,8,9,10,11] )
ACEo_SInd_Fu_E, dts_over_SInd_Fu_E, dts_under_SInd_Fu_E = \
    ttJ.sst_composite_1 (basin=7, BACE_stack=BACE_stack, ts_stack=ts_stack_E,
                    landf=landf, months=SHseason,dont_scale = dont_scale) #[1,2,3,4,5,10,11,12] )
ACEo_NEPac_Fu_E, dts_over_NEPac_Fu_E, dts_under_NEPac_Fu_E = \
    ttJ.sst_composite_1 (basin=4, BACE_stack=BACE_stack, ts_stack=ts_stack_E,
                    landf=landf, months=NHseason,dont_scale = dont_scale) #[5,6,7,8,9,10,11] )
ACEo_xNEPac_Fu_E, dts_over_xNEPac_Fu_E, dts_under_xNEPac_Fu_E = \
    ttJ.sst_composite_1 (basin=2, BACE_stack=sBACE_stack, ts_stack=ts_stack_E,
                    landf=landf, months=NHseason,dont_scale = dont_scale) #[5,6,7,8,9,10,11] )


In [ ]:
#         -1.       0.        1.         2            3             4              5            6 
bnames=['None','N Atlantic', 'S Atlantic' , 'NW Pacific', 'NE Pacific', 'SW Pacific','N Indian','  S Indian']

AnaMo = [ allyear, NHseason,    SHseason,    NHseason,     NHseason,      SHseason,    NHseason,   SHseason ]

In [ ]:
NEPac4tup=(85,100,190,220)
NEPac4tup=(85,100,140,220)

NAtl4tup=(110,120,240,270)


In [ ]:
#STOP

In [ ]:
basins = IBT.basin_outlines(Mediterranean=False,xNEPacific=True)
print(f"You'll be working with xNEPacific sub-basin")


In [ ]:
############
# FANCY .... 
# Plot SSTs associated high and low ACE in selected basin
############

buffer_distance = -1.  # For basin drawing. Adjust as needed

levels=2*np.linspace(-1,1,num=21)

fig = plt.figure(figsize=(25, 18))

c1=np.asarray( (NEPac4tup) )
c2=np.asarray( (NAtl4tup)  )


projMap  = ccrs.PlateCarree(central_longitude=180.)
projData = ccrs.PlateCarree()

cbshrink=0.7
nxplo=2
nyplo=4
n=0
xpanlab=-0.02
ypanlab= 1.1

levels=0.2*np.linspace(-1,1,num=21)

#########################################
# Start w/ N Atlantic
#########################################

n=n+1

basin=1
BasinPoly = basins[basin].xy 

Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 , projection=projMap )
co1 = ax1.contourf( lon,lat, dts_over_NAtl_Fu_E, levels=levels,cmap='bwr' ,transform=projData )
bxw = ax1.plot(  lon[[ c1[2],c1[2],c1[3],c1[3],c1[2] ]] ,  lat[[c1[0],c1[1],c1[1],c1[0],c1[0]]] ,color='black',transform=projData)
ax1.add_feature(cartopy.feature.LAND, zorder=1, edgecolor='black')    #ax.set_xlim(100,280)
plot_BasinPoly(ax1,BasinPoly)
ax1.set_xticks([])
ax1.set_xlabel('')
ax1.set_yticks([])
ax1.set_ylabel('')
ax1.set_title(f"Monthly N Atlantic ACE > {ACEo_NAtl_Fu_E:.1f}",fontsize=16)
cb1 = plt.colorbar(co1,shrink=cbshrink)
ax1.set_extent( [180.-140,180+170, -50, 50] , crs=projData)
ipan=n-1
ax1.text(xpanlab,ypanlab, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')



n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 , projection=projMap )
co1 = ax1.contourf( lon,lat, dts_under_NAtl_Fu_E, levels=levels,cmap='bwr' ,transform=projData )
bxw = ax1.plot(  lon[[ c1[2],c1[2],c1[3],c1[3],c1[2] ]] ,  lat[[c1[0],c1[1],c1[1],c1[0],c1[0]]] ,color='black',transform=projData)
ax1.add_feature(cartopy.feature.LAND, zorder=1, edgecolor='black')    #ax.set_xlim(100,280)
plot_BasinPoly(ax1,BasinPoly)
ax1.set_xticks([])
ax1.set_xlabel('')
ax1.set_yticks([])
ax1.set_ylabel('')
ax1.set_title(f"Monthly N Atlantic ACE < {ACEo_NAtl_Fu_E:.1f}",fontsize=16)
cb1 = plt.colorbar(co1,shrink=cbshrink)
ax1.set_extent( [180.-140,180+170, -50, 50] , crs=projData)
ipan=n-1
ax1.text(xpanlab,ypanlab, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

#########################################
# Switch to S Indian
#########################################
basin=7
BasinPoly = basins[basin].xy 

levels=0.5*np.linspace(-1,1,num=21)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 , projection=projMap )
co1 = ax1.contourf( lon,lat, dts_over_SInd_Fu_E, levels=levels,cmap='bwr' ,transform=projData )
#bxw = ax1.plot(  lon[[ c1[2],c1[2],c1[3],c1[3],c1[2] ]] ,  lat[[c1[0],c1[1],c1[1],c1[0],c1[0]]] ,color='black',transform=projData)
ax1.add_feature(cartopy.feature.LAND, zorder=1, edgecolor='black')    #ax.set_xlim(100,280)
plot_BasinPoly(ax1,BasinPoly)
ax1.set_xticks([])
ax1.set_xlabel('')
ax1.set_yticks([])
ax1.set_ylabel('')
ax1.set_title(f"Monthly S Indian ACE > {ACEo_SInd_Fu_E:.1f}",fontsize=16)
cb1 = plt.colorbar(co1,shrink=cbshrink)
ax1.set_extent( [180.-140,180+170, -50, 50] , crs=projData)
ipan=n-1
ax1.text(xpanlab,ypanlab, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 , projection=projMap )
co1 = ax1.contourf( lon,lat, dts_under_SInd_Fu_E, levels=levels,cmap='bwr',transform=projData  )
#bxw = ax1.plot(  lon[[ c1[2],c1[2],c1[3],c1[3],c1[2] ]] ,  lat[[c1[0],c1[1],c1[1],c1[0],c1[0]]] ,color='black',transform=projData)
ax1.add_feature(cartopy.feature.LAND, zorder=1, edgecolor='black')    #ax.set_xlim(100,280)
plot_BasinPoly(ax1,BasinPoly)
ax1.set_xticks([])
ax1.set_xlabel('')
ax1.set_yticks([])
ax1.set_ylabel('')
ax1.set_title(f"Monthly S Indian ACE < {ACEo_SInd_Fu_E:.1f}",fontsize=16)
cb1 = plt.colorbar(co1,shrink=cbshrink)
ax1.set_extent( [180.-140,180+170, -50, 50] , crs=projData)
ipan=n-1
ax1.text(xpanlab,ypanlab, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')


#########################################
# Switch to NE Pacific
#########################################
basin=4
BasinPoly = basins[basin].xy 
levels=0.5*np.linspace(-1,1,num=21)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 , projection=projMap )
co1 = ax1.contourf( lon,lat, dts_over_NEPac_Fu_E, levels=levels,cmap='bwr' ,transform=projData )
#bxw = ax1.plot(  lon[[ c1[2],c1[2],c1[3],c1[3],c1[2] ]] ,  lat[[c1[0],c1[1],c1[1],c1[0],c1[0]]] ,color='black',transform=projData)
ax1.add_feature(cartopy.feature.LAND, zorder=1, edgecolor='black')    #ax.set_xlim(100,280)
plot_BasinPoly(ax1,BasinPoly)
ax1.set_xticks([])
ax1.set_xlabel('')
ax1.set_yticks([])
ax1.set_ylabel('')
ax1.set_title(f"Monthly NE Pacific ACE > {ACEo_NEPac_Fu_E:.1f}",fontsize=16)
cb1 = plt.colorbar(co1,shrink=cbshrink)
ax1.set_extent( [180.-140,180+170, -50, 50] , crs=projData)
ipan=n-1
ax1.text(xpanlab,ypanlab, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 , projection=projMap  )
co1 = ax1.contourf( lon,lat, dts_under_NEPac_Fu_E, levels=levels,cmap='bwr' ,transform=projData )
#bxw = ax1.plot(  lon[[ c1[2],c1[2],c1[3],c1[3],c1[2] ]] ,  lat[[c1[0],c1[1],c1[1],c1[0],c1[0]]] ,color='black',transform=projData)
ax1.add_feature(cartopy.feature.LAND, zorder=1, edgecolor='black')    #ax.set_xlim(100,280)
plot_BasinPoly(ax1,BasinPoly)
ax1.set_xticks([])
ax1.set_xlabel('')
ax1.set_yticks([])
ax1.set_ylabel('')
ax1.set_title(f"Monthly NE Pacific ACE < {ACEo_NEPac_Fu_E:.1f}",fontsize=16)
cb1 = plt.colorbar(co1,shrink=cbshrink)
ax1.set_extent( [180.-140,180+170, -50, 50] , crs=projData)
ipan=n-1
ax1.text(xpanlab,ypanlab, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

#########################################
# Switch to xtreme NE Pacific
#########################################
basin=8
BasinPoly = basins[basin].xy 
levels=0.2*np.linspace(-1,1,num=21)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 , projection=projMap )
co1 = ax1.contourf( lon,lat, dts_over_xNEPac_Fu_E, levels=levels,cmap='bwr' ,transform=projData )
#bxw = ax1.plot(  lon[[ c1[2],c1[2],c1[3],c1[3],c1[2] ]] ,  lat[[c1[0],c1[1],c1[1],c1[0],c1[0]]] ,color='black',transform=projData)
ax1.add_feature(cartopy.feature.LAND, zorder=1, edgecolor='black')    #ax.set_xlim(100,280)
plot_BasinPoly(ax1,BasinPoly)
ax1.set_xticks([])
ax1.set_xlabel('')
ax1.set_yticks([])
ax1.set_ylabel('')
ax1.set_title(f"Monthly xNE Pacific ACE > {ACEo_xNEPac_Fu_E:.1f}",fontsize=16)
cb1 = plt.colorbar(co1,shrink=cbshrink)
ax1.set_extent( [180.-140,180+170, -50, 50] , crs=projData)
ipan=n-1
ax1.text(xpanlab,ypanlab, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 , projection=projMap  )
co1 = ax1.contourf( lon,lat, dts_under_xNEPac_Fu_E, levels=levels,cmap='bwr' ,transform=projData )
#bxw = ax1.plot(  lon[[ c1[2],c1[2],c1[3],c1[3],c1[2] ]] ,  lat[[c1[0],c1[1],c1[1],c1[0],c1[0]]] ,color='black',transform=projData)
ax1.add_feature(cartopy.feature.LAND, zorder=1 ,edgecolor='black')    #ax.set_xlim(100,280)
plot_BasinPoly(ax1,BasinPoly)
ax1.set_xticks([])
ax1.set_xlabel('')
ax1.set_yticks([])
ax1.set_ylabel('')
ax1.set_title(f"Monthly xNE Pacific ACE < {ACEo_xNEPac_Fu_E:.1f}",fontsize=16)
cb1 = plt.colorbar(co1,shrink=cbshrink)
ax1.set_extent( [180.-140,180+170, -50, 50] , crs=projData)
ipan=n-1
ax1.text(xpanlab,ypanlab, f"{chr(97 +ipan)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

Pu.save_link(png_filename = "NEPac-NAtl-ana.png")


In [ ]:
# Create time series
ys,yn,xw,xe = NEPac4tup #80,110,150,200
ts_NEPacX = np.average( np.average( ts_stack[ :,:,:,ys:yn,xw:xe] , axis=4) , axis=3 )
print( f"NE Pac Region limits. Lon: {lon[xw]-360:.2f}, {lon[xe]-360:.2f} Lat: {lat[ys]:.2f}, {lat[yn]:.2f} ")


ys,yn,xw,xe = NAtl4tup #100,130,225,260
ts_NAtlX = np.average( np.average( ts_stack[ :,:,:,ys:yn,xw:xe] , axis=4) , axis=3 )
print( f"N Atl Region limits. Lon: {lon[xw]-360:.2f}, {lon[xe]-360:.2f} Lat: {lat[ys]:.2f}, {lat[yn]:.2f} ")


ts_Diff = 1.*ts_NEPacX - ts_NAtlX


In [ ]:
# Create time series
ys,yn,xw,xe = NEPac4tup #80,110,150,200
ts_NEPacE = np.average( np.average( ts_stack_E[ :,:,:,ys:yn,xw:xe] , axis=4) , axis=3 )
print( f"NE Pac Region limits. Lon: {lon[xw]-360:.2f}, {lon[xe]-360:.2f} Lat: {lat[ys]:.2f}, {lat[yn]:.2f} ")


ys,yn,xw,xe = NAtl4tup #100,130,225,260
ts_NAtlE = np.average( np.average( ts_stack_E[ :,:,:,ys:yn,xw:xe] , axis=4) , axis=3 )
print( f"N Atl Region limits. Lon: {lon[xw]-360:.2f}, {lon[xe]-360:.2f} Lat: {lat[ys]:.2f}, {lat[yn]:.2f} ")


ts_Diff_E = 1.*ts_NEPacE - ts_NAtlE


In [ ]:
print(ts_NEPacX.flatten().shape)
print(BACE_stack[:,7,:,:].flatten().shape)

In [ ]:
pdf_range = [(-3,4),(0,200.)] 

nbins=[30,28]
nbins=[15,14]
x= ts_NEPacX.flatten()
Hrx=[]
for b in np.arange(8):
    y=BACE_stack[:,b,:,:].flatten()
    Hr,X0,Y0=np.histogram2d(x, y, bins=nbins   ,range=pdf_range )
    Hr  =  Hr/np.size(x) #np.sum(Hr)
    Hrx.append(Hr)

XX=(X0[1:]+X0[:-1])/2
YY=(Y0[1:]+Y0[:-1])/2
#XX=X0[:-1]
#YY=Y0[:-1]
y=BACE_stack[:,1,:,:].flatten()


In [ ]:
nepac_terciles,tercile_limits = pd.qcut( pd.Series( ts_NEPacE.flatten() ), 3,retbins=True )
print(tercile_limits)

In [ ]:
pdf_range = [(-3,4),(0,200.)] 

nbins=[30,28]
nbins=[15,14]
x= ts_NEPacE.flatten()
Hrx_E=[]
for b in np.arange(8):
    y=BACE_stack[:,b,:,:].flatten()
    Hr,X0,Y0=np.histogram2d(x, y, bins=nbins   ,range=pdf_range )
    Hr  =  Hr/np.size(x) #np.sum(Hr)
    Hrx_E.append(Hr)

XX=(X0[1:]+X0[:-1])/2
YY=(Y0[1:]+Y0[:-1])/2


In [ ]:
from matplotlib.colors import LogNorm


In [ ]:
lev=np.array( [0.002,0.005,0.01,0.02,0.03,0.05,0.07,0.1] )/10
print(lev)

cmap=Pu.step_cmap(levels=lev,cmapN='gist_ncar')
nxplo,nyplo=1,1

fig = plt.figure(figsize=(10, 6))

xr0,xr1= pdf_range[0][0],pdf_range[0][1] # -6,2 #-4,2.7 #-3,3.7

Axes0 = Pu.axes_def(n=1,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes0 )
#hoo=ax1.pcolor(XX,YY, Hrx_E[1].T/Hrx_E[1][:,1] , cmap='gist_ncar',
#              norm=LogNorm(vmin=0.002, vmax=2.1) )
hoo=ax1.pcolor(XX,YY, Hrx_E[1].T , cmap='gist_ncar',
              norm=LogNorm(vmin=0.0002, vmax=0.1) )
#hoo=ax1.pcolor(XX,YY, Hrx[1].T,cmap=cmap )
sca=ax1.scatter( ts_NEPacE.flatten() , BACE_stack[:,1,:,:].flatten() ,color='black',s=10)
cbr=plt.colorbar(hoo, location='right')
cbr.set_label( label='Probability',size=16)
cbr.ax.tick_params(labelsize=12)

ax1.set_xlim(xr0,xr1)
ax1.set_ylim(pdf_range[1] ) #0,160)
ax1.set_xlabel(f"Eq Pacific T anomaly (K)",fontsize=16)
ax1.set_ylabel(f"N Atlantic monthly ACE",fontsize=16)


Pu.save_link(png_filename = "ACE-NEPac_TS-joint.png")

In [ ]:
pdf_range = [(-3,4),(0,200.)] 


nbins=[3,14]
x= ts_NEPacE.flatten()
Hrx_E=[]
for b in np.arange(8):
    y=BACE_stack[:,b,:,:].flatten()
    Hr,X0,Y0=np.histogram2d(x, y, bins=[tercile_limits,14]   ,range=pdf_range )
    Hr  =  Hr/np.size(x) #np.sum(Hr)
    Hrx_E.append(Hr)

XX=(X0[1:]+X0[:-1])/2
YY=(Y0[1:]+Y0[:-1])/2
print(X0)

In [ ]:
def ffoo(n):
    foo=f"EqE Pac SST': {X0[n]:.2f} to {X0[n+1]:.2f}"
    return foo


In [ ]:
# Get some pastel shades for the colors
colors = plt.cm.BuPu(np.linspace(0.2, 0.5, 3))
colors = plt.cm.Blues(np.linspace(0.2, 0.8, 3))


Hss = Hrx_E[1].T #/Hrx_E[1][:,1]

n=0
plt.bar( YY[1:]+n*2,Hss[1:,n],color=colors[2-n],width=5,label=ffoo(n))
n=1
plt.bar( YY[1:]+n*2,Hss[1:,n],color=colors[2-n],width=5,label=ffoo(n))
n=2
plt.bar( YY[1:]+n*2,Hss[1:,n],color=colors[2-n],width=5,label=ffoo(n))
plt.legend()
plt.yscale('log')
Pu.save_link(png_filename = "ACE-by-SST-tercile.png")

In [ ]:
pdf_range = [(-3,4),(0,200.)] 

#nbins=[30,28]
nbins=[3,50_000]
#x= ts_NEPacX.flatten()
Hrx=[]
for b in np.arange(8):
    #for b in np.arange(start=1,stop=2):
    #monix = np.array(AnaMo[b])-1 
    monix = np.arange(12) #np.array(AnaMo[b])-1 
    xtmp= ts_NEPacE[:,:, monix ]
    ytmp =BACE_stack[:,:,:, monix ]
    print( len(monix) )
    nMo=len(monix)
    x=xtmp.flatten()
    y=ytmp[:,b,:,:].flatten()   #BACE_stack[:,b,:,:].flatten()
    Hr,X0,Y0=np.histogram2d(x, y, bins=[tercile_limits,50_000]   ,range=pdf_range )
    Hr  =  (nMo/12)* Hr/np.size(x) #np.sum(Hr)
    Hrx.append(Hr)

XX=(X0[1:]+X0[:-1])/2
YY=(Y0[1:]+Y0[:-1])/2


In [ ]:
print( np.average( y ))

In [ ]:
Hrx[1].shape

In [ ]:
b=1
print( f" Presnt day mean {12*np.average( BACE_pd_stack[:,b,:,:].flatten() ):.2f} ")
print( np.average( 12*BACE_stack[:,b,:,:].flatten() ) )
for n in np.arange( start=0,stop=3):
    Scale_pdf =12. #len(AnaMo[b])/12.
    print( f" PDF {ffoo(n)}, ACE = {  Scale_pdf *np.sum(Hrx[b][n,:]*YY[:]) / np.sum(Hrx[b][n,:]) :.2f}" )
    #print( f" PDF {ffoo(n)}, ACE = {  Scale_pdf *np.sum(Hrx[b][n,1:]*YY[1:]) :.2f}" )


In [ ]:
plt.plot(ts_NEPacE[1,:,:].flatten()) 


In [ ]:
plt.plot( BACE_stack[1,1,:,:].flatten() )

In [ ]:
print( np.shape(BACE_stack))
print( np.shape(ts_NEPacE))
print( np.shape(ts_stack_O))
print( np.shape(ts_stack_A))


In [ ]:
ys,yn,xw,xe = NEPac4tup #80,110,150,200
ts_NEPacOO = np.average( np.average( ts_stack_O[ :,:,:,ys:yn,xw:xe] , axis=4) , axis=3 )


In [ ]:
print( np.shape(ts_NEPacOO))


In [ ]:
plt.figure( figsize=(10,6))
plt.plot(ts_NEPacOO[1,:,:].flatten() , label='NE Pac SST (SST2) ') 
plt.plot(ts_NEPacE[1,:,:].flatten() +300., label='Deseasonalized (-ensemble mean) +300K') 
plt.plot(0*ts_NEPacE[1,:,:].flatten() +300., '--',color='orange') 
plt.xlabel( 'Months after 2070/1',fontsize=16)
plt.ylabel( 'SSTs (K)',fontsize=16)
plt.legend()

In [ ]:
plt.figure( figsize=(10,6))
plt.plot( BACE_stack[1,1,:,:].flatten(), label='NE Atlantic ACE (SST2) ') 
plt.xlabel( 'Months after 2070/1',fontsize=16)
plt.ylabel( 'ACE',fontsize=16)
plt.legend()